# makemore: becoming a backprop ninja

## 2024 08 20 practice

## 2024 11 26 finish the video 

## Question : how should I use x += y or x = x + y

- 感觉是没啥区别，我一般会写成x=x+y

## 2025 03 20

终于完成了exercise1的所有任务
- 计算梯度时的难点：某个layer被重复使用，梯度要累加；矩阵乘法的梯度是最基础的梯度，通过torch.einsum实现; embedding indexing的梯度


### Exercise1

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures


In [3]:
# read in all the words
words = open('makemore/names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [6]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [7]:
vocab_size

27

In [8]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [9]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [10]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean() # plug out all the correct index probabilities and average

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way # important !!!! otherwise no gradient for the non-leaft tensors
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3064, grad_fn=<NegBackward0>)

In [11]:
logprobs.shape

torch.Size([32, 27])

In [12]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n
# 只有这N个value参与了计算，其他的位置没有参与到计算 
# no matter how log probs is confidently wrong , the gradient of loss function wrt. logprobs is -1/n on the correct class

In [13]:
dlogprobs_probs = probs ** -1 
dprobs = dlogprobs * dlogprobs_probs # here a uniform distribution and a extreme value distribution will make different

In [14]:
dcounts_sum_inv = torch.einsum("ij,ij->i",dprobs,counts)[:,None] # dl/db_i = \sum_j dl/dc_ij * dc_ij/db_i from gradient chain rule

In [15]:
dcounts_sum = (-1*counts_sum**(-2))*dcounts_sum_inv

In [16]:
# dcounts should be solved by chain rule
# exp : z = yx, y = 2x; dz = ydx + xdy , dy = 2 dx 

# forward
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv

dcounts_1 = torch.einsum("ij,i->ij",dprobs,counts_sum_inv[...,0]) # first parts of gradients
dcounts_2 = torch.ones_like(counts) * dcounts_sum
dcounts = dcounts_1 + dcounts_2

# Q : take two things into consideration, uniform softmax and a extreme value softmax, what will happen?

In [17]:
# forward
# counts = norm_logits.exp() ; the derivative of exp is exp x 
with torch.no_grad():
    dnorm_logits = dcounts * counts

In [18]:
# forward
# norm_logits = logits - logit_maxes # subtract max for numerical stability

# dlogit_maxes = -1 * torch.einsum("ij->i",dnorm_logits)
with torch.no_grad():
    dlogit_maxes  = -1 * torch.sum(dnorm_logits,1,keepdim=True)


In [19]:
# forward
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
with torch.no_grad():
    # dlogits for two parts
    dlogits1 = torch.ones_like(logits) * dnorm_logits
    dlogits2 = torch.zeros_like(logits)
    # find max_positions
    max_positions = logits.argmax(1,keepdim=False)
    dlogits2[range(n),max_positions] = 1 * dlogit_maxes[...,0]
    dlogits = dlogits1 + dlogits2
    # we have dlogit_maxes 

In [20]:
# IN MLP, we don't need tensor / tensor, we can all get scalar / tensor

# forward
# Linear layer 2
# logits = h @ W2 + b2 # output layer
# don't compute the local gradient
# just give the gradient of the loss wrt. weight and linear

with torch.no_grad():
    dh =  torch.einsum("kj,ij->ki",dlogits,W2) 
    # dh = dlogits @ W2.T
    db2 = torch.sum(dlogits,dim=0,keepdim=True).squeeze()
    # dW2 = torch.einsum("kj,ki->ij",dlogits,h)  may cause numerical bugs
    dW2 = h.T @ dlogits 

In [21]:
# forward

# hpreact = bngain * bnraw + bnbias
# h = torch.tanh(hpreact) # hidden layer
with torch.no_grad():
    dhpreact =  dh * (1-h**2)
    dbngain = torch.sum(dhpreact*bnraw,dim=0,keepdim=True) 
    dbnbias = torch.sum(dhpreact,dim=0,keepdim=True)
    dbnraw = dhpreact * bngain

In [22]:
# forward
# emb = C[Xb] # embed the characters into vectors
# embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# # Linear layer 1
# hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# # BatchNorm layer
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
with torch.no_grad():
    dbnvar_inv = torch.sum(dbnraw * bndiff,dim=0,keepdim=True)
    dbnvar = dbnvar_inv * (-0.5) * (bnvar + 1e-5)**(-1.5)
    dbndiff2 = (1/(n-1) * dbnvar).repeat(32,1)
    dbndiff = 2 * bndiff * dbndiff2 + dbnraw * bnvar_inv
    dbnmeani = -1 * dbndiff.sum(dim=0,keepdim=True)   
    dhprebn = dbndiff + 1/n * dbnmeani
    dembcat = torch.einsum("ik,jk->ij",dhprebn,W1) 
    dW1 = torch.einsum("ij,ik->jk",embcat,dhprebn)
    db1 = dhprebn.sum(dim=0)
    demb = dembcat.reshape(emb.shape)
    dC = torch.zeros_like(C)
    for batch_idx in range(Xb.shape[0]):
        for char_idx in range(Xb.shape[1]):
            tensor_idx = Xb[batch_idx,char_idx].item()
            dC[tensor_idx] += demb[batch_idx,char_idx]

In [23]:
# dC = torch.zeros_like(C)
# for batch_idx in range(Xb.shape[0]):
#     for char_idx in range(Xb.shape[1]):
#         tensor_idx = Xb[batch_idx,char_idx].item()
#         dC[tensor_idx] += demb[batch_idx,char_idx]

In [24]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n
dprobs = dlogprobs * (probs ** -1) # 对于真正label，如果概率p非常低，那么1/p就会非常大，因此logp会boost这个梯度
dcounts_sum_inv = (dprobs * counts).sum(dim=1,keepdim=True)   # two steps for pytorch : 1. 广播成同样的形状 2.对于同样的形状计算梯度
# element-wise梯度：dprobs * counts 是loss对于广播的之后的矩阵梯度
# 广播、replication梯度：在广播的维度相加


cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
bngain          | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
bnbias          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnraw   

### Exercise2

将logits - CrossEntropy写到一起

In [25]:

# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.3063859939575195 diff: 0.0


In [26]:
def cross_entropy_gradient(logits,y):
    # single batch gradient 
    grad = torch.exp(logits) / torch.sum(torch.exp(logits))
    grad[y] -= 1
    return grad

In [27]:

# backward pass
# dlogits = torch.zeros_like(logits)
# for batch_idx in range(n):
#     dlogits[batch_idx] = cross_entropy_gradient(logits[batch_idx], Yb[batch_idx]) / n


# 一种更优雅的方式, crossentropy的梯度就是prob, 在label处为prob - 1
dlogits = F.softmax(logits, dim=1)
dlogits[range(n), Yb] -= 1
dlogits /= n

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: True  | maxdiff: 6.51925802230835e-09


In [28]:
dlogits.shape

torch.Size([32, 27])

In [29]:
logits.shape

torch.Size([32, 27])

In [30]:
logits = torch.randn(10)
logits.requires_grad = True
y = 2
probs = F.softmax(logits, dim=0)
logprobs = probs.log()
loss = -logprobs[y].mean()
torch.autograd.grad(loss, logits)


(tensor([ 0.0613,  0.0975, -0.9668,  0.1645,  0.1916,  0.0447,  0.1374,  0.1183,
          0.1025,  0.0489]),)

In [31]:
cross_entropy_gradient(logits,y)

tensor([ 0.0613,  0.0975, -0.9668,  0.1645,  0.1916,  0.0447,  0.1374,  0.1183,
         0.1025,  0.0489], grad_fn=<CopySlices>)

In [32]:
dlogits_computation = torch.exp(logits) / torch.sum(torch.exp(logits))
dlogits_computation[y] -= 1
dlogits_computation


tensor([ 0.0613,  0.0975, -0.9668,  0.1645,  0.1916,  0.0447,  0.1374,  0.1183,
         0.1025,  0.0489], grad_fn=<CopySlices>)

## Exercise 3

将batch normalization整合到一起

In [33]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [34]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


### Exercise 4

将forward与backward, update写到一起

In [35]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

  # kick off optimization
  for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    for p in parameters:
      p.grad = None
    #loss.backward() # use this for correctness comparisons, delete it later!

    # manual backprop! #swole_doge_meme
    # -----------------
    dlogits = F.softmax(logits, 1)
    dlogits[range(n), Yb] -= 1
    dlogits /= n
    # 2nd layer backprop
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0)
    # tanh
    dhpreact = (1.0 - h**2) * dh
    # batchnorm backprop
    dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
    dbnbias = dhpreact.sum(0, keepdim=True)
    dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
    # 1st layer
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = dhprebn.sum(0)
    # embedding
    demb = dembcat.view(emb.shape)
    dC = torch.zeros_like(C)
    for k in range(Xb.shape[0]):
      for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]
    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p, grad in zip(parameters, grads):
      #p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
      p.data += -lr * grad # new way of swole doge TODO: enable

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

  #   if i >= 100: # TODO: delete early breaking when you're ready to train the full net
  #     break

12297
      0/ 200000: 3.7555
  10000/ 200000: 2.1554
  20000/ 200000: 2.3344
  30000/ 200000: 2.4711
  40000/ 200000: 1.9529
  50000/ 200000: 2.3565
  60000/ 200000: 2.4089
  70000/ 200000: 2.0273
  80000/ 200000: 2.2614
  90000/ 200000: 2.1883
 100000/ 200000: 1.9724
 110000/ 200000: 2.3216
 120000/ 200000: 1.9399
 130000/ 200000: 2.4385
 140000/ 200000: 2.3242
 150000/ 200000: 2.1719
 160000/ 200000: 1.9338
 170000/ 200000: 1.8476
 180000/ 200000: 2.1449
 190000/ 200000: 1.9264


In [38]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [41]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.071352243423462
val 2.1112256050109863


In [42]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # ------------
      # forward pass:
      # Embedding
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # ------------
      # Sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

mona.
mayah.
seel.
ndhonalee.
thruthadrie.
cadelyn.
elin.
shi.
jenleigh.
esmana.
sepheimhotalin.
shreer.
shirael.
kindreelynn.
novana.
ubelled.
ryyah.
fael.
yuma.
mys.


In [120]:
import torch
def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    """torch.repeat_interleave(x, dim=2, repeats=n_rep) from Llamma3"""
    bs, slen, n_kv_heads, head_dim = x.shape
    if n_rep == 1:
        return x
    return (
        x[:, :, :, None, :]
        .expand(bs, slen, n_kv_heads, n_rep, head_dim)
        .reshape(bs, slen, n_kv_heads * n_rep, head_dim)
        )

In [24]:
x = torch.randn(1, 2, 4, 2)
y = repeat_kv(x, 2)

In [25]:
x

tensor([[[[-0.9775, -1.0789],
          [ 0.1624, -1.8405],
          [ 0.9507,  0.3011],
          [-1.2406, -0.5837]],

         [[-1.1423,  1.7853],
          [ 0.0409,  1.1610],
          [-0.2402,  1.9578],
          [ 0.7019,  2.1591]]]])

In [26]:
y

tensor([[[[-0.9775, -1.0789],
          [-0.9775, -1.0789],
          [ 0.1624, -1.8405],
          [ 0.1624, -1.8405],
          [ 0.9507,  0.3011],
          [ 0.9507,  0.3011],
          [-1.2406, -0.5837],
          [-1.2406, -0.5837]],

         [[-1.1423,  1.7853],
          [-1.1423,  1.7853],
          [ 0.0409,  1.1610],
          [ 0.0409,  1.1610],
          [-0.2402,  1.9578],
          [-0.2402,  1.9578],
          [ 0.7019,  2.1591],
          [ 0.7019,  2.1591]]]])

In [27]:
z

tensor([[[[-0.7903,  2.4070, -0.4276,  0.0947, -1.2497],
          [-0.7903,  2.4070, -0.4276,  0.0947, -1.2497],
          [ 0.8955, -0.0571, -1.2833, -0.0313, -0.2610],
          [ 0.8955, -0.0571, -1.2833, -0.0313, -0.2610],
          [ 1.2286, -0.2659, -0.5508,  0.8652,  0.6643],
          [ 1.2286, -0.2659, -0.5508,  0.8652,  0.6643],
          [ 0.2125, -0.7300, -2.1901, -1.4154,  0.2888],
          [ 0.2125, -0.7300, -2.1901, -1.4154,  0.2888]],

         [[ 0.5943,  0.8673,  1.7449, -0.9035, -0.3915],
          [ 0.5943,  0.8673,  1.7449, -0.9035, -0.3915],
          [ 0.8661, -1.0370, -0.2133,  0.4721,  0.2251],
          [ 0.8661, -1.0370, -0.2133,  0.4721,  0.2251],
          [ 0.0478, -0.0921, -1.5668,  1.3102,  0.3000],
          [ 0.0478, -0.0921, -1.5668,  1.3102,  0.3000],
          [ 0.9285,  1.5418,  0.3743,  2.2138, -0.0551],
          [ 0.9285,  1.5418,  0.3743,  2.2138, -0.0551]],

         [[-0.5614,  0.1855,  0.5033,  0.4404,  0.4352],
          [-0.5614,  0.1855

In [28]:
z = torch.repeat_interleave(x, dim=2, repeats=2)

In [29]:
z

tensor([[[[-0.9775, -1.0789],
          [-0.9775, -1.0789],
          [ 0.1624, -1.8405],
          [ 0.1624, -1.8405],
          [ 0.9507,  0.3011],
          [ 0.9507,  0.3011],
          [-1.2406, -0.5837],
          [-1.2406, -0.5837]],

         [[-1.1423,  1.7853],
          [-1.1423,  1.7853],
          [ 0.0409,  1.1610],
          [ 0.0409,  1.1610],
          [-0.2402,  1.9578],
          [-0.2402,  1.9578],
          [ 0.7019,  2.1591],
          [ 0.7019,  2.1591]]]])

In [58]:
AA_FREQ = {'A': 0.07421620506799341,
 'R': 0.05161448614128464,
 'N': 0.044645808512757915,
 'D': 0.05362600083855441,
 'C': 0.02468745716794485,
 'Q': 0.03425965059141602,
 'E': 0.0543119256845875,
 'G': 0.074146941452645,
 'H': 0.026212984805266227,
 'I': 0.06791736761895376,
 'L': 0.09890786849715096,
 'K': 0.05815568230307968,
 'M': 0.02499019757964311,
 'F': 0.04741845974228475,
 'P': 0.038538003320306206,
 'S': 0.05722902947649442,
 'T': 0.05089136455028703,
 'W': 0.013029956129972148,
 'Y': 0.03228151231375858,
 'V': 0.07291909820561925}

In [35]:
# make ESM2 backprop


L = 256
hidden = 1280

pssm = torch.randn(L, 20)
pssm.requires_grad = True
W = torch.randn(20, hidden)
W.requires_grad = True

In [36]:
one_hot = pssm.argmax(1,keepdim=False)
one_hot = F.one_hot(one_hot,20).float()
one_hot.requires_grad = True
activation = one_hot @ W
loss = activation.sum()
# real gradient
loss.backward()

In [14]:
dl_dactivation = torch.ones_like(activation)

In [42]:
dl_dactivation.shape

torch.Size([256, 1280])

In [54]:
done_hot = torch.einsum("kj,ij->ki",dl_dactivation, W)
dW = torch.einsum("kj, ki->ij",dl_dactivation,one_hot)

In [50]:
done_hot 

tensor([[ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610],
        [ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610],
        [ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610],
        ...,
        [ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610],
        [ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610],
        [ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610]],
       grad_fn=<ViewBackward0>)

In [39]:
one_hot.grad

tensor([[ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610],
        [ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610],
        [ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610],
        ...,
        [ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610],
        [ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610],
        [ 57.4189,  17.2741,  -7.4375,  ...,   7.2317,  46.7600, -11.4610]])

In [30]:
activation.grad

/tmp/ipykernel_12563/4104473693.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1711403380481/work/build/aten/src/ATen/core/TensorBody.h:489.)
  activation.grad


In [40]:
one_hot

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)